In [9]:
import concurrent.futures
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import datetime
import time

# Define the headers
headers = ["Transaction ID", "Transaction Type", "Transaction Date", "Transaction Status", "Transferring Registry", "Transferring Account Type", "Transferring Account Name", "Transferring Account Identifier", "Transferring Account Holder", "Acquiring Registry", "Acquiring Account Type", "Acquiring Account Name", "Acquiring Account Identifier", "Acquiring Account Holder","Nb of units","Option","Page","Time","Time taken","Time taken formatted"]

# Create a directory to store the files
directory = "threads"
if not os.path.exists(directory):
    os.makedirs(directory)

# Define a function to fetch the data from a page
def fetch_data(page_number):
    url = 'https://ec.europa.eu/clima/ets/transaction.do?languageCode=fr&startDate=&endDate=&transactionStatus=4&fromCompletionDate=&toCompletionDate=&transactionID=&transactionType=-1&suppTransactionType=-1&originatingRegistry=-1&destinationRegistry=-1&originatingAccountType=-1&destinationAccountType=-1&originatingAccountIdentifier=&destinationAccountIdentifier=&originatingAccountHolder=&destinationAccountHolder=&currentSortSettings=&backList=%3CBack&resultList.currentPageNumber={}'.format(page_number)
    response = requests.get(url)
    # soup = BeautifulSoup(response.content, 'lxml')
    # soup = BeautifulSoup(response.content.decode('UTF-8'), 'lxml')
    soup = BeautifulSoup(response.text, 'lxml')

    all_tables = soup.find_all('table', {'class': 'bordertb'})
    if len(all_tables) >= 2:
        outer_table = all_tables[1]
        inner_table = outer_table.find('table', {'id': 'tblTransactionSearchResult'})
        if inner_table is not None:
            rows = []
            for row in inner_table.find_all('tr')[2:]:
                cells = row.find_all('td')
                rows.append([val.text.strip() for val in cells[:len(cells)-1]])
            rows = [row for row in rows if any(cell.strip() != "" for cell in row)]
            return rows
    return None
start_time = time.time()
# Create a thread pool with 16 workers
            
            
            
            
            
            
#             df = pd.DataFrame(result, columns=headers)

#             # Save the data to a file
#             file_name = f"transactions/transactions_{i+1}.csv"
#             df.to_csv(file_name, index=False)

# # Merge all the files into a single CSV file
# merged_file = f"transactions/merged_transactions.csv"
# df = pd.concat([pd.read_csv(f) for f in os.listdir(directory) if f.endswith('.csv')], ignore_index=True)
# df.to_csv(merged_file, index=False)

# print("I am done scraping the data and merged the files into a single CSV file")


In [12]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    # Use the map function to fetch the data for each page number in parallel   
    

    results = executor.map(fetch_data, range(2,10)) #0-4230-8460-12690-16920-21150-25380-29610-33840-38070-42300-46530-50760-55342
    # print results type   
    print(type(results))
# results is a data frame, so we can iterate over it and fuse each result to a dataframe
    #create dataframe
    finaldf = pd.DataFrame()
    for i, result in enumerate(results):
        df = pd.DataFrame(results, columns=headers) 
        file_name = f"{directory}/transactions_{i}.csv"
        finaldf = pd.concat([finaldf,df])
    finaldf.to_csv('finaldest1.csv', index=False)




# Merge all the files into a single CSV file
# merged_file = f"{directory}/merged_transactions.csv"
# with open(merged_file, 'w') as outfile:
#     for file_name in os.listdir(directory):
#         if file_name.startswith("transactions_"):
#             with open(f"{directory}/{file_name}") as infile:
#                 outfile.write(infile.read())

end_time = time.time()
# total_time = end_time - start_time
time_taken = end_time - start_time
time_taken_formatted = str(datetime.timedelta(seconds=time_taken))
print("Time taken: {}".format(time_taken_formatted))


print("I am done scraping the data and merged it into a single file")

df.head()
# df.to_csv('thread1.csv', index=False)

<class 'generator'>
(2, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Transaction ID                   2 non-null      object
 1   Transaction Type                 2 non-null      object
 2   Transaction Date                 2 non-null      object
 3   Transaction Status               2 non-null      object
 4   Transferring Registry            2 non-null      object
 5   Transferring Account Type        2 non-null      object
 6   Transferring Account Name        2 non-null      object
 7   Transferring Account Identifier  2 non-null      object
 8   Transferring Account Holder      2 non-null      object
 9   Acquiring Registry               2 non-null      object
 10  Acquiring Account Type           2 non-null      object
 11  Acquiring Account Name           2 non-null      object
 12  Acquiring Ac

,Transaction ID,Transaction Type,Transaction Date,Transaction Status,Transferring Registry,Transferring Account Type,Transferring Account Name,Transferring Account Identifier,Transferring Account Holder,Acquiring Registry,Acquiring Account Type,Acquiring Account Name,Acquiring Account Identifier,Acquiring Account Holder,Nb of units,Option,Page,Time,Time taken,Time taken formatted
0,"[EU520648, 10-2, 2019-04-30 19:02:55.43, Compl...","[EU520730, 10-2, 2019-04-30 18:22:24.609, Comp...","[EU520679, 10-2, 2019-04-30 18:17:03.961, Comp...","[EU520354, 10-0, 2019-04-30 17:59:41.157, Comp...","[EU520725, 10-2, 2019-04-30 17:54:51.281, Comp...","[EU520351, 10-0, 2019-04-30 17:54:30.24, Compl...","[DE123705, 4-0, 2019-04-30 17:48:05.517, Compl...","[EU520717, 10-2, 2019-04-30 17:41:54.296, Comp...","[EU520533, 10-2, 2019-04-30 17:39:51.73, Compl...","[EU520322, 10-0, 2019-04-30 17:12:44.056, Comp...","[EU519952, 10-0, 2019-04-30 17:09:00.919, Comp...","[EU520311, 10-0, 2019-04-30 17:04:45.043, Comp...","[EU520681, 10-2, 2019-04-30 16:57:18.192, Comp...","[EU520309, 10-0, 2019-04-30 16:51:38.085, Comp...","[EU520606, 10-2, 2019-04-30 16:51:01.555, Comp...","[EU520305, 10-0, 2019-04-30 16:46:33.152, Comp...","[EU520719, 10-2, 2019-04-30 16:46:29.146, Comp...","[EU520303, 10-0, 2019-04-30 16:37:48.493, Comp...","[EU520289, 10-0, 2019-04-30 16:32:30.984, Comp...","[EU520716, 10-2, 2019-04-30 16:30:12.035, Comp..."
1,"[EU520648, 10-2, 2019-04-30 19:02:55.43, Compl...","[EU520730, 10-2, 2019-04-30 18:22:24.609, Comp...","[EU520679, 10-2, 2019-04-30 18:17:03.961, Comp...","[EU520354, 10-0, 2019-04-30 17:59:41.157, Comp...","[EU520725, 10-2, 2019-04-30 17:54:51.281, Comp...","[EU520351, 10-0, 2019-04-30 17:54:30.24, Compl...","[DE123705, 4-0, 2019-04-30 17:48:05.517, Compl...","[EU520717, 10-2, 2019-04-30 17:41:54.296, Comp...","[EU520533, 10-2, 2019-04-30 17:39:51.73, Compl...","[EU520322, 10-0, 2019-04-30 17:12:44.056, Comp...","[EU519952, 10-0, 2019-04-30 17:09:00.919, Comp...","[EU520311, 10-0, 2019-04-30 17:04:45.043, Comp...","[EU520681, 10-2, 2019-04-30 16:57:18.192, Comp...","[EU520309, 10-0, 2019-04-30 16:51:38.085, Comp...","[EU520606, 10-2, 2019-04-30 16:51:01.555, Comp...","[EU520305, 10-0, 2019-04-30 16:46:33.152, Comp...","[EU520719, 10-2, 2019-04-30 16:46:29.146, Comp...","[EU520303, 10-0, 2019-04-30 16:37:48.493, Comp...","[EU520289, 10-0, 2019-04-30 16:32:30.984, Comp...","[EU520716, 10-2, 2019-04-30 16:30:12.035, Comp..."
